## Imports

In [4]:
import os
import numpy as np
import pandas as pd


## Functions

In [5]:
def is_nan(nan):
    return nan != nan


def read_excel_data(excel_path,
                    delete_cols=[],
                    delete_sourcename=[]):
    df = pd.read_excel(excel_path)
    header_name_list = list(df.head())
    all_data = np.array(df)
    all_data = np.concatenate([np.reshape(header_name_list, (1, -1)), all_data], axis=0)
    print('all_data shape: {}'.format(np.shape(all_data)))

    row, col = np.shape(all_data)
    source_name_list, header_name_list, label_list, data = [], [], [], [] # np.zeros((row, col-2))

    for i in range(row):
        row_data = []
        for j in range(col):
            if i == 0:
                if 'Source_Name' == all_data[i, j]:
                    source_name_index = j
                    print('source_name_index: {}'.format(source_name_index))
                    continue
                elif 'Class_Xiao' == all_data[i, j]:
                    label_name_index = j
                    print('label_name_index: {}'.format(label_name_index))
                    continue
                header_name_list.append(all_data[i, j])
            else:
                if source_name_index == j:
                    source_name = all_data[i, j]
                    if source_name in delete_sourcename:
                        break
                    else:
                        source_name_list.append(source_name)
                elif label_name_index == j:
                    label_list.append(all_data[i, j])
                else:
                    try:
                        row_data.append(float(all_data[i, j]))
                    except:
                        print('error in ({}, {}) -> {}'.format(i, j, all_data[i, j]))
        if len(row_data) > 0:
            data.append(row_data)

    return source_name_list, header_name_list, label_list, np.array(data)



def pick_wanted_samples(source_name_list, label_list, data, wanted_labels):
    wanted_data_dict = {}
    wanted_sourcename_dict = {}
    for i in range(len(label_list)):
        if not is_nan(label_list[i]):
            label = label_list[i].lower()
            if label in wanted_labels:
                if label not in wanted_data_dict.keys():
                    wanted_data_dict[label] = [data[i]]
                    wanted_sourcename_dict[label] = [source_name_list[i]]
                else:
                    wanted_data_dict[label].append(data[i])
                    wanted_sourcename_dict[label].append(source_name_list[i])
    return wanted_sourcename_dict, wanted_data_dict



def process_nan_values(input_data):
    output_data = np.copy(input_data)
    row, col = np.shape(input_data)
    for j in range(col):
        col_data = input_data[:, j]
        col_data_ = []
        for i in range(row):
            d = input_data[i, j]
            if not is_nan(d):
                col_data_.append(d)
        mean_value = np.mean(col_data_)
        for i in range(row):
            d = output_data[i, j]
            if is_nan(d):
                output_data[i, j] = mean_value
        if len(col_data_) != row:
            print('col: {}, mean: {}, has NaN {}'.format(j, mean_value, row-len(col_data_)))
    
    checked = True
    for i in range(row):
        for j in range(col):
            if is_nan(output_data[i, j]):
                checked = False
    
    if checked:
        return output_data
    else:
        print('still has NaN values')
        return None


def process_original_file(src_file, target_file, wanted_labels_list):
    save_data_dict = {} # the data to be saved in target_file

    # read the src_file, remove some cols and rows
    source_name_list, header_name_list, label_list, data = read_excel_data(src_file)
    wanted_sourcename_dict, wanted_data_dict_ = pick_wanted_samples(source_name_list, label_list, data, wanted_labels=wanted_labels_list)
    
    # build the headers
    save_data_dict['source_name'] = []
    save_data_dict['label'] = []
    for header_name in header_name_list:
        save_data_dict[header_name] = []

    
    wanted_data_dict = {}
    for k, v in wanted_data_dict_.items():
        wanted_data_dict[k] = process_nan_values(np.array(v))
        print(k, np.shape(wanted_data_dict[k]))
    
    
    for label_name, source_data in wanted_data_dict.items():
        row, col = np.shape(source_data)
        assert col == len(header_name_list)
        assert row == len(wanted_sourcename_dict[label_name])
        for i in range(row):
            save_data_dict['source_name'].append(wanted_sourcename_dict[label_name][i])
            save_data_dict['label'].append(label_name)
            for j in range(col):
                header_name = header_name_list[j]
                d = source_data[i, j]
                save_data_dict[header_name].append(d)
    
    save_data_dict = pd.DataFrame(save_data_dict)
    save_data_dict.to_excel(target_file)





## Load and clean Data

In [6]:
src_file = r'datasets\4FGL_DR3_Data_A.xlsx'
target_file = r'datasets\Dataset_A.xlsx'
wanted_labels_list = ['agn', 'non-agn', 'un']
process_original_file(src_file, target_file, wanted_labels_list)


all_data shape: (6660, 18)
source_name_index: 0
label_name_index: 9
un (2291, 16)
agn (3809, 16)
col: 2, mean: 2.2397136912783155e-10, has NaN 1
col: 3, mean: 2.344321295519713, has NaN 1
col: 4, mean: 0.06511966051225807, has NaN 1
col: 5, mean: 18.91627668297491, has NaN 1
col: 6, mean: 0.07948483414293907, has NaN 1
col: 7, mean: 6.081420805236918, has NaN 1
non-agn (559, 16)


In [7]:
src_file = r'datasets\4FGL_DR3_Data_B.xlsx'
target_file = r'datasets\Dataset_B.xlsx'
wanted_labels_list = ['bcu', 'bll', 'fsrq']
process_original_file(src_file, target_file, wanted_labels_list)

all_data shape: (3744, 18)
source_name_index: 0
label_name_index: 9
bcu (1493, 16)
bll (1456, 16)
fsrq (794, 16)
